# Agents Coursework

In [ ]:
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/ngcm/training-public/master/ipython_notebook_styles/ngcmstyle.css'
HTML(url=css_file)

## Predator-prey

Building on the CSA model for a flock of agents in lab 2, we now want to study how behaviour changes if a different type of agent is introduced: a predator.

The predator agent will be faster than the agents in the flock. Again it will move at constant speed, which will be $50\%$ greater than that of the flock members. However, the predator will be less nimble. Whilst the flock members will be able to change heading every step (which will always be $\Delta t = 0.05$), the predator will only be able to change heading every $4$ steps.

The predator will obey the same cohesion and alignment rules as the flock, but will not try and maintain separation. That is, the predator utility function will be

\begin{equation}
  f_{\text{predator}} \left( \theta ; {\bf z}, {\bf z}_i, {\bf v}_i \right) = C \cos(\theta - \theta_z) + A \cos(\theta - \theta_V).
\end{equation}

If the predator gets within a distance $r_{\text{eat}}$ of a flock member, the flock member is "eaten" and removed from the simulation. The predator "sees" the whole flock: its locality radius is $r_l = \infty$.

The utility function of members of the flock remains, as in lab 2,

\begin{equation}
  f_{\text{flock}} \left( \theta ; {\bf z}, {\bf z}_i, {\bf v}_i \right) = C \cos(\theta - \theta_z) + A \cos(\theta - \theta_V) - S \frac{\cos(\theta - \theta_{z_{\text{min}}})}{\Delta z_{\text{min}}^2 }.
\end{equation}

The flock members will avoid the predator in the same way they avoid other members of the flock - via the separation term.

In [ ]:
%matplotlib inline
import math
from math import atan2
from numpy import cos, sin, pi
import numpy as np
from matplotlib import pyplot, animation
from JSAnimation import IPython_display
from multiprocessing import Pool
from numba import jit

from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (14,7)

import scipy
from scipy.optimize import minimize

from __future__ import division

The class describing the movement of one flock agent

In [ ]:
class Agent(object):
    """
    Class that defines the properties of an agent based on the CSA model
    """
    
    def __init__(self, location, velocity, C=1, A=5, S=0.1):
        # Initialise the Agent object properties.
        self.loc = np.array(location)    # Stores the position components
        self.v = np.array(velocity)      # Stores the velocity components
        self.C = C                       # Stores the cohesion value
        self.A = A                       # Stores the alignment value
        self.S = S                       # Stores the separation value
    
    def step(self, dt):
        # Move the agent according to a given time step 'dt' and its velocity
        self.loc += self.v * dt
    #@jit
    def steer(self, neighbours, predators=[]):
        """
        Change the direction of the agent according to the position
        of its neighbours and that of the predator(s).
        """
        # Compute only if the agent has neighbours
        N = len(neighbours)
        if N == 0:
            return
        min_sep = 100.0
        # Initialise vectors
        diff_avg_loc = np.zeros_like(self.loc)
        min_sep_direction = diff_avg_loc.copy()
        avg_v = diff_avg_loc.copy()
        # Compute the average positions/velocities
        # and the neighbour that is nearest to the agent
        for ag in neighbours:
            diff_avg_loc += ag.loc - self.loc
            avg_v += ag.v
            sep = np.linalg.norm(ag.loc - self.loc)
            if sep < min_sep:
                min_sep = sep
                min_sep_direction = ag.loc - self.loc
        avg_v /= N
        diff_avg_loc /= N
        v = np.linalg.norm(self.v)
        
        # Compute theta values
        th = atan2(self.v[1], self.v[0])
        th_z = atan2(diff_avg_loc[1], diff_avg_loc[0])
        th_V = atan2(avg_v[1], avg_v[0])
        th_min_z = atan2(min_sep_direction[1], min_sep_direction[0])
        minz = np.linalg.norm(min_sep_direction)
        
        C = self.C
        A = self.A
        S = self.S
        
        if predators:
            # If there are no predators do not compute
            # their effect on separation
            sep = 100.0
            for pred in predators:
                sep_pred = pred.loc - self.loc
                p_dist = np.linalg.norm(sep_pred)
                if p_dist < sep:
                    p_dist_min = p_dist
                    sep_pred_min = sep_pred
            sep_pred /= len(predators)
            th_pred = atan2(sep_pred_min[1], sep_pred_min[0])
            predator_effect = lambda theta: cos(theta - th_pred) / p_dist_min**2
        else:
            predator_effect = lambda theta: 0.0
        
        utility = lambda theta: -(C * cos(theta - th_z)+\
                                A * cos(theta - th_V)-\
                                S * (cos(theta - th_min_z) / minz**2 +\
                                    predator_effect(theta)))
        res = minimize(utility, th, bounds=[[-2*pi, 2*pi]])
        th_new = res.x[0]
        self.v = np.array([v * cos(th_new), v * sin(th_new)])

In [ ]:
class Predator(object):
    
    def __init__(self, location, velocity, C=1, A=0.1):
        # Initialise the Agent object properties.
        self.loc = np.array(location)    # Stores the position components
        self.v = np.array(velocity)      # Stores the velocity components
        self.C = C                       # Stores the cohesion value
        self.A = A                       # Stores the alignment value
    
    def step(self, dt):
        # Take one step
        self.loc += self.v * dt
    #@jit
    def steer(self, neighbours):
        # Ensure the predator has some prey to catch
        N = len(neighbours)
        if N == 0:
            return
        # Initialise vectors
        diff_avg_loc = np.zeros_like(self.loc)
        min_sep_direction = diff_avg_loc.copy()
        avg_v = diff_avg_loc.copy()
        # Compute neighbour averages
        for ag in neighbours:
            diff_avg_loc += ag.loc - self.loc
            avg_v += ag.v
        avg_v /= N
        diff_avg_loc /= N
        v = np.linalg.norm(self.v)
        
        # Compute theta values
        th = atan2(self.v[1], self.v[0])
        th_z = atan2(diff_avg_loc[1], diff_avg_loc[0])
        th_V = atan2(avg_v[1], avg_v[0])
        
        utility = lambda theta: -(self.C * cos(theta - th_z) + self.A * cos(theta - th_V))
        res = minimize(utility, th, bounds=[[-2*pi, 2*pi]])
        th_new = res.x[0]
        self.v = np.array([v * cos(th_new), v * sin(th_new)])

The class describing the movement of the whole flock.

In [ ]:
class Hunt(object):
    def __init__(self, locations, velocities, p_loc=[], p_vel=[], s=0.1, rl=1):
        """
        Initialises the 'hunt' object variables and verifies inputs.
        """
        assert len(locations) > 1,\
        "Please ensure you have entered the (x, y) positions of at least one agent"
        assert len(velocities) > 1,\
        "Please ensure you have entered the (x, y) velocities of at least one agent"
        try:
            assert ((not (p_loc or p_vel)) or (p_loc and p_vel)), \
            "Please ensure that the velocity is inserted when the\
            positions are given for the predator and vice-versa"
        except ValueError:
            assert len(p_loc) > 1, "Ensure that the predator location is given correctly"
            assert len(p_vel) > 1, "Ensure that the predator velocity is given correctly"
        
        # Note that the list of agents are the prey
        self.rl = rl        # The radius in which the agents 'see' each other
        self.pstep = 0      # The number of steps the predator has taken
        self.agents = []    # A list to store agent objects
        self.predators = [] # A list to store predator objects
        # Transform 1-D inputs to 2-D
        try:
            np.shape(locations)[1]
        except IndexError:
            locations = np.array(locations)[:, np.newaxis].T
        try:
            np.shape(velocities)[1]
        except IndexError:
            velocities = np.array(velocities)[:, np.newaxis].T
        # Compute predator step only if locations and velocities are available
        
        if (len(p_loc) > 1) and (len(p_vel) > 1):
            try:
                np.shape(p_loc)[1]
            except IndexError:
                p_loc = np.array(p_loc)[:, np.newaxis].T
            try:
                np.shape(p_vel)[1]
            except IndexError:
                p_vel = np.array(p_vel)[:, np.newaxis].T

            # Store a list of predator objects
            for loc, vel in zip(p_loc, p_vel):
                self.predators.append(Predator(loc, vel))
        # Store a list of agent objects
        for loc, vel in zip(locations, velocities):
            self.agents.append(Agent(loc, vel, S=s))
        self.predators = np.array(self.predators)
        self.agents = np.array(self.agents)
    
    
    #@jit
    def step(self, dt):
        if len(self.agents)==0:
            return
        # Move the flock
        for i, agent in enumerate(self.agents):
            index = np.linalg.norm(self.locations(i) - np.array([agent.loc]).T, axis=0) < self.rl
            index[i] = False
            agent.steer(np.array(self.agents)[index], self.predators)
        for agent in self.agents:
            agent.step(dt)
        
        # Move the predators if they exist and remove agents within radius
        if self.predators:
            for predator in self.predators:
                if self.pstep == 4:
                    predator.steer(self.agents)
                    self.pstep = 0
                predator.step(dt)
                self.pstep += 1
                eat_index = np.linalg.norm(self.locations() - np.array([predator.loc]).T, axis=0) > 0.3
                self.agents = self.agents[eat_index]
            
    
    def locations(self, i=None):
        # Returns a 2xN matrix with the (x, y) locations of 
        # each agent (prey), where N corresponds to the total
        # number of agents
        x = np.zeros(len(self.agents))
        y = x.copy()
        for j, agent in enumerate(self.agents):
            if not i == j:
                x[j], y[j] = agent.loc
        return np.vstack((x, y))
    
    def p_locations(self):
        # Returns a 2xN matrix with the (x, y) locations of each predator, 
        # where N corresponds to the total number of predators
        x = np.zeros(len(self.predators))
        y = x.copy()
        for j, predator in enumerate(self.predators):
            x[j], y[j] = predator.loc
        return np.vstack((x, y))
    
    def velocities(self):
        # Returns a 2xN matrix with the velocities of each agent (prey), 
        # where N corresponds to the total number of agents
        vx = np.zeros(len(self.agents))
        vy = vx.copy()
        for i, agent in enumerate(self.agents):
            vx[i], vy[i] = agent.v
        return np.vstack((vx, vy))
    
    def p_velocities(self):
        # Returns a 2xN matrix with the (x, y) velocities of each predator, 
        # where N corresponds to the total number of predators
        vx = np.zeros(len(self.predators))
        vy = vx.copy()
        for i, predator in enumerate(self.predators):
            vx[i], vy[i] = predator.v
        return np.vstack((vx, vy))
    
    def average_location(self):
        # Computes the average flock location
        return np.mean(self.locations(), axis=1)
    
    def average_velocity(self):
        # Computes the average velocity of the flock
        return np.mean(self.velocities(), axis=1)
    
    def average_width(self):
        # Returns the average width of the flock
        locs = self.locations()
        average_loc = self.average_location()
        distances = np.linalg.norm(locs-average_loc[:,np.newaxis], axis=0)
        return np.mean(distances)

In [ ]:
def test_suite():
    """Simple test suite to see if the classes behave correctly"""
    def test_predator_chase_1():
        dt = 0.1
        bird_pos = [0.0, 0.0]
        bird_v = [0.0, 1.0]
        hawk_pos = [2.0, 2.0]
        hawk_v = [0.0, -2.0]
        hunt = Hunt(bird_pos, bird_v, hawk_pos, hawk_v, s=0.1)
        i = 0
        while len(hunt.agents) > 0:
            hunt.step(dt)
            i+=1
        if i < 20:
            return True
        else:
            return False
    
    def test_agent_travel():
        dt = 0.1
        bird_pos = np.array([[0.0, 0.0], [0.5, 0.8]])
        bird_v = np.array([[0.0, 1.0], [0.0, 1.0]])
        hunt = Hunt(bird_pos, bird_v, s=0.001)
        i = 0
        while np.linalg.norm(hunt.agents[0].loc - hunt.agents[1].loc) > 0.6:
            hunt.step(dt)
            i+=1
            if i>100:
                break
        if i < 100:
            return True
        else:
            return False
    
    if test_predator_chase_1():
        print("Test 1: The one agent one predator test has ended succesfully. The predator has caught the prey")
    if test_agent_travel():
        print("Test 2: The two agent test has ended succesfully. The agents cluster.")

In [ ]:
test_suite()

In [ ]:
def flock_animation(flock, dt, frames=10, xlim=(-0.1, 5), ylim=(-0.1, 5)):
    # First evolve
    
    locations = [flock.locations()]
    p_loc = [flock.p_locations()]
    ave_width = [flock.average_width()]
    times = np.arange(0.0, frames*dt, dt)
    for i in range(frames):
        flock.step(dt)
        #print(flock.predators)
        locations.append(flock.locations())
        p_loc.append(flock.p_locations())
        ave_width.append(flock.average_width())
    #print(len(flock.agents))
    max_width = max(ave_width)
    min_width = min(ave_width)
    d_width = max_width - min_width
    
    fig = pyplot.figure()
    ax1 = fig.add_subplot(121)
    ax1.set_xlim(xlim[0], xlim[1])
    ax1.set_ylim(ylim[0], ylim[1])
    points, = ax1.plot([], [], 'ro')
    pointsP, = ax1.plot([], [], 'bo')
    ax1.set_xlabel("$x$")
    ax1.set_ylabel("$y$")
    ax2 = fig.add_subplot(122)
    width, = ax2.plot([], [], 'b-')
    ax2.set_xlabel("$t$")
    ax2.set_ylabel("Average width of flock")
    ax2.set_xlim(0.0, dt*frames)
    ax2.set_ylim(min_width-0.1*d_width, max_width+0.1*d_width)
    fig.tight_layout()

    def init():
        points.set_data([], [])
        pointsP.set_data([], [])
        width.set_data([], [])
        return (points, width)

    def animate(i):
        points.set_data(locations[i][0,:], locations[i][1,:])
        pointsP.set_data(p_loc[i][0,:], p_loc[i][1,:])
        width.set_data(times[:i+1], ave_width[:i+1])
        return (points, width)

    return animation.FuncAnimation(fig, animate, init_func=init, interval=100, frames=frames, blit=True)

Start with a flock of $100$ agents in an array within $[0, 5]^2$, all moving "due north". Start the predator at $(15, 15)$ with velocity "due west":

In [ ]:
x = np.linspace(0, 4.5, 10)
[locations_flock_x, locations_flock_y] = np.meshgrid(x, x)
locations_flock_x[:,::2] += 0.25
locations_flock_y[:,::2] += 0.25
locations = np.vstack((np.ravel(locations_flock_x), np.ravel(locations_flock_y))).T
velocities = np.zeros_like(locations)
velocities[:,1] = 1.0
location_predator = np.array([15.0, 15.0])
velocity_predator = np.array([-2.0, 0.0])
pyplot.plot(locations[:,0], locations[:,1], 'ro', label='Flock')
pyplot.quiver(locations[:,0], locations[:,1], velocities[:,0], velocities[:,1], pivot='mid', units='xy')
pyplot.plot(15, 15, 'b*', markersize=16, label='Predator')
pyplot.quiver(location_predator[0], location_predator[1], velocity_predator[0], velocity_predator[1], pivot='mid', units='xy')
pyplot.legend(loc='lower right')
pyplot.show()

Fix the cohesion and alignment parameters for both the predator to be $C = 1, A = 10^{-2}$, and for the flock to be $C = 1, A = 5$. Show how the separation parameter $S$ for the flock changes the number of the flock that survive in evolutions up to $t=50$. A (small!) range of $S$ between $10^{-3}$ and $100$ should be considered.

In [ ]:
hunt = Hunt(locations, velocities, location_predator, velocity_predator, s=0.001)
flock_animation(hunt, 0.05, 1000, xlim=(-30.0,30.0), ylim=(-30.0,30.0))

In [ ]:
hunt = Hunt(locations, velocities, location_predator, velocity_predator, s=1)
flock_animation(hunt, 0.05, 1000, xlim=(-30.0,30.0), ylim=(-30.0,30.0))

The code below computes the effect of separation on the number of agents surviving the predator attack

In [ ]:
dt = 0.05
sep = []
alive = []
"""
# Parallel implementation
def run(arg):
    flock, dt, steps, S = arg
    for i in np.arange(steps):
        flock.step(dt)
    return len(flock.agents)
pool = Pool()
args = []
for S in np.logspace(-3, 2, 16):
    args.append((hunt, dt, 1000, S))
results = pool.map(run, args)

"""
for S in np.logspace(-3, 2, 16):
    hunt = Hunt(locations, velocities, location_predator, velocity_predator, s=S)
    for i in np.arange(1000):
        hunt.step(dt)
        if len(hunt.agents)==0:
            break
    alive.append(len(hunt.agents))
    sep.append(S)

In [ ]:
figure = pyplot.figure()
ax = figure.add_subplot(111)
ax.semilogx(sep, alive)
ax.set_xlabel('Separation value')
ax.set_ylabel('Alive members')
ax.set_title(r'Number of flock members alive after $50s$')
#ax.grid(b=True, which='major', color='r', linestyle='-')
ax.grid(b=True, which='minor', axis='x', color='r', linestyle='--')
ax.grid(b=True, which='major', axis='y', color='r', linestyle='--')

To avoid running the funtion above, the figure below was produced. It shows how the number of members that survive the predator attack varies with the size of the separation value.

For low separation none of the agents survive, while at higher values almost all of them survive. The reason for this is that the predator heads for the geometrical center of the flock and the flock spreads out thus causing the predator to constantly switch the direction of movement and becoming 'indecisive' as to which group of agents to follow. This can be seen in the two animations above.

<img src="hunt.png">